In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados processados
dados = pd.read_parquet("TMDB_movie_dataset_v11(com correçao).parquet")

# Manter apenas as colunas especificadas
dados = dados[['vote_average', 'vote_count', 'revenue', 'runtime', 'budget', 'popularity', 'lucratividade']]

# Dropar linhas onde a coluna 'lucratividade' tem o valor 'lucratividade_indisponivel'
dados_filtrados = dados[dados['lucratividade'] != 'lucratividade_indisponivel']

# Verificar se as linhas foram dropadas
print(f"Linhas antes de dropar: {len(dados)}")
print(f"Linhas depois de dropar: {len(dados_filtrados)}")
print(f"Ocorrências de 'lucratividade_indisponivel' após dropar: {dados_filtrados['lucratividade'].value_counts().get('lucratividade_indisponivel', 0)}")

# Mostrar a porcentagem de cada categoria no DataFrame original
print("\nPorcentagem de cada categoria no DataFrame original:")
print(dados['lucratividade'].value_counts(normalize=True) * 100)

# Amostragem estratificada para garantir a representatividade das categorias
dados_sample = dados_filtrados.groupby('lucratividade', group_keys=False).apply(lambda x: x.sample(min(len(x), 5000), random_state=42))

# Mostrar a porcentagem de cada categoria na amostra
print("\nPorcentagem de cada categoria na amostra:")
print(dados_sample['lucratividade'].value_counts(normalize=True) * 100)

# Separar as features e o target
X = dados_sample.drop(columns=['lucratividade'])
y = dados_sample['lucratividade']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Treinar o modelo K-Nearest Neighbors
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Fazer previsões
y_pred = knn_model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Linhas antes de dropar: 1131886
Linhas depois de dropar: 14218
Ocorrências de 'lucratividade_indisponivel' após dropar: 0

Porcentagem de cada categoria no DataFrame original:
lucratividade
lucratividade_indisponivel    98.743866
prejuizo                       0.550762
super_alta                     0.197811
muito_alta                     0.181732
media                          0.119800
baixo                          0.079072
alta                           0.075538
muito_baixo                    0.051419
Name: proportion, dtype: float64

Porcentagem de cada categoria na amostra:
lucratividade
prejuizo       38.508934
super_alta     17.244301
muito_alta     15.842575
media          10.443623
baixo           6.893099
alta            6.585028
muito_baixo     4.482440
Name: proportion, dtype: float64


C:\Users\merci\AppData\Local\Temp\ipykernel_14948\3586811659.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dados_sample = dados_filtrados.groupby('lucratividade', group_keys=False).apply(lambda x: x.sample(min(len(x), 5000), random_state=42))


Accuracy: 0.5517905275317674
Classification Report:
               precision    recall  f1-score   support

        alta       0.28      0.32      0.29       171
       baixo       0.25      0.24      0.24       179
       media       0.32      0.29      0.31       271
  muito_alta       0.48      0.51      0.50       412
 muito_baixo       0.12      0.05      0.07       116
    prejuizo       0.69      0.79      0.74      1000
  super_alta       0.70      0.55      0.62       448

    accuracy                           0.55      2597
   macro avg       0.41      0.39      0.40      2597
weighted avg       0.54      0.55      0.54      2597

